In [1]:
from Bio import SeqIO
import pandas as pd
import duckdb
import re
import plotly.graph_objects as go
import numpy as np


def parse_taxonomy(description):
    """Extrae información taxonómica de la descripción"""
    tax_match = re.search(r'd([^;]+);p([^;]+);c([^;]+);o([^;]+);f([^;]+);g([^;]+);s__([^\s]+)', description)
    if tax_match:
        return {
            'domain': tax_match.group(1),
            'phylum': tax_match.group(2),
            'class': tax_match.group(3),
            'order': tax_match.group(4),
            'family': tax_match.group(5),
            'genus': tax_match.group(6),
            'species': tax_match.group(7)
        }
    return dict.fromkeys(['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species'])

def fna_to_dataframe(fna_file):
    """Convierte archivo FNA a DataFrame"""
    records = []
    
    for record in SeqIO.parse(fna_file, "fasta"):
        sequence = str(record.seq)
        tax_info = parse_taxonomy(record.description)
        
        records.append({
            'sequence_id': record.id,
            'sequence': sequence,
            'sequence_length': len(sequence),
            'gc_content': (sequence.count("G") + sequence.count("C")) / len(sequence) * 100,
            'domain': tax_info['domain'],
            'phylum': tax_info['phylum'],
            'class': tax_info['class'],
            'order': tax_info['order'],
            'family': tax_info['family'],
            'genus': tax_info['genus'],
            'species': tax_info['species']
        })
    
    return pd.DataFrame(records)

def export_to_fasta(df, output_file):
    """Exporta DataFrame a archivo FASTA con información seleccionada"""
    with open(output_file, 'w') as f:
        for _, row in df.iterrows():
            header = f">{row['sequence_id']} length={row['sequence_length']} GC={row['gc_content']} genus={row['genus']}"
            f.write(f"{header}\n{row['sequence']}\n")

def read_exported_fasta(fasta_file):
    """Lee el archivo FASTA exportado"""
    records = []
    
    for record in SeqIO.parse(fasta_file, "fasta"):
        header_info = record.description.split()
        
        # Extraer información del header
        sequence_id = header_info[0]
        length = int(header_info[1].split('=')[1])
        gc = float(header_info[2].split('=')[1])
        genus = header_info[3].split('=')[1]
        
        records.append({
            'sequence_id': sequence_id,
            'sequence': str(record.seq),
            'sequence_length': length,
            'gc_content': gc,
            'genus': genus
        })
    
    return pd.DataFrame(records)

In [ ]:
df = fna_to_dataframe('datos/datos_originales_ncbi.fna')

: 

In [ ]:
columnas=["domain",'phylum','class','order','family','genus','species']

for c in columnas:
    print(f"{c}: {df[c].nunique()}")

domain: 1
phylum: 171
class: 458
order: 1553
family: 3884
genus: 16117
species: 16117


In [4]:
for i,v in enumerate(df['genus']):
    df.loc[i,'genus']=v.replace('__', '')

In [5]:
dfcopia=df.copy()

In [6]:
df=duckdb.sql("""
select * from df
           where sequence_length<1600 and sequence_length>1300
""").to_df()

In [7]:
columnas=["domain",'phylum','class','order','family','genus','species']

for c in columnas:
    print(f">>>{c}: {df[c].nunique()}")

>>>domain: 1
>>>phylum: 155
>>>class: 385
>>>order: 1227
>>>family: 2858
>>>genus: 10576
>>>species: 10576


In [8]:
# def plot_sequence_length_enhanced(df, col, color_scheme='Viridis', 
#                                  show_stats=True, show_cumulative=False,
#                                  bin_size=None, text_size_factor=1.3):
#     """
#     Enhanced function to visualize DNA sequence length distribution
    
#     Parameters:
#     - df: DataFrame containing the data
#     - col: Column name with sequence lengths
#     - color_scheme: Color scheme for the chart ('Viridis', 'Plasma', 'Turbo', etc.)
#     - show_stats: Show descriptive statistics
#     - show_cumulative: Show cumulative distribution
#     - bin_size: Bin size (for very large sequences)
#     - text_size_factor: Factor to increase all text sizes
#     """
#     # Extract length data
#     sequence_lengths = df[col].copy()
    
#     # Calculate statistics
#     stats = {
#         'Mean': np.mean(sequence_lengths),
#         'Median': np.median(sequence_lengths),
#         'Mode': sequence_lengths.mode().iloc[0] if not sequence_lengths.mode().empty else None,
#         'Min': sequence_lengths.min(),
#         'Max': sequence_lengths.max(),
#         'Std. Dev': np.std(sequence_lengths)
#     }
    
#     # Determine if we should use bins for widely dispersed distributions
#     unique_lengths = len(sequence_lengths.unique())
#     use_histogram = unique_lengths > 50 or (stats['Max'] - stats['Min']) > 100
    
#     if bin_size is None:
#         # Calculate bin_size automatically to show more columns
#         # We want at least 30 bins for a more detailed histogram
#         range_length = stats['Max'] - stats['Min']
#         bin_size = max(1, int(range_length / 40))  # Ensure at least 40 bins for detailed view
    
#     # Create base figure
#     fig = go.Figure()
    
#     # Count sequence length frequencies
#     sequence_counts = df[col].value_counts().reset_index()
#     sequence_counts.columns = ['Length', 'Frequency']
#     sequence_counts = sequence_counts.sort_values('Length')
    
#     # Calculate cumulative distribution
#     total = sequence_counts['Frequency'].sum()
#     sequence_counts['Percentage'] = sequence_counts['Frequency'] * 100 / total
#     sequence_counts['Cumulative'] = sequence_counts['Frequency'].cumsum() * 100 / total
    
#     # Add bar chart with larger bars and no colorbar
#     fig.add_trace(go.Bar(
#         x=sequence_counts['Length'],
#         y=sequence_counts['Frequency'],
#         name='Frequency',
#         marker=dict(
#             color=sequence_counts['Frequency'],
#             colorscale=color_scheme,
#             showscale=False,  # Remove the colorbar
#         ),
#         hovertemplate='Length: %{x}<br>Frequency: %{y}<br>Percentage: %{text:.2f}%<extra></extra>',
#         text=sequence_counts['Percentage']
#     ))
    
#     # Add cumulative distribution line
#     if show_cumulative:
#         fig.add_trace(go.Scatter(
#             x=sequence_counts['Length'],
#             y=sequence_counts['Cumulative'],
#             mode='lines+markers',
#             name='% Cumulative',
#             line=dict(color='rgba(219, 64, 82, 0.8)', width=3),
#             marker=dict(size=8),
#             yaxis='y2',
#             hovertemplate='Length: %{x}<br>Cumulative: %{y:.2f}%<extra></extra>'
#         ))
    
#     # Add reference lines for important statistics
#     if show_stats:
#         # Add vertical line for mean
#         fig.add_vline(x=stats['Mean'], line_width=2, line_dash="dash", line_color="green",
#                      annotation=dict(
#                          text=f"Mean: {stats['Mean']:.2f}",
#                          font=dict(size=14 * text_size_factor, color="green"),
#                          xanchor="right",
#                          yanchor="top"
#                      ))
        
#         # Add vertical line for median
#         fig.add_vline(x=stats['Median'], line_width=2, line_dash="dash", line_color="red",
#                      annotation=dict(
#                          text=f"Median: {stats['Median']}",
#                          font=dict(size=14 * text_size_factor, color="red"),
#                          xanchor="left",
#                          yanchor="top"
#                      ))
    
#     # Create statistics text block with larger font
#     stats_text = "<br>".join([
#         f"<b style='font-size:{14 * text_size_factor}px'>Descriptive Statistics:</b>",
#         f"<span style='font-size:{13 * text_size_factor}px'>Mean: {stats['Mean']:.2f}</span>",
#         f"<span style='font-size:{13 * text_size_factor}px'>Median: {stats['Median']}</span>",
#         f"<span style='font-size:{13 * text_size_factor}px'>Mode: {stats['Mode']}</span>",
#         f"<span style='font-size:{13 * text_size_factor}px'>Min: {stats['Min']}, Max: {stats['Max']}</span>",
#         f"<span style='font-size:{13 * text_size_factor}px'>Std. Dev: {stats['Std. Dev']:.2f}</span>",
#         f"<span style='font-size:{13 * text_size_factor}px'>Total sequences: {len(sequence_lengths)}</span>"
#     ])
    
#     # Customize the layout with larger fonts
#     fig.update_layout(
#         title={
#             'text': 'DNA Sequence Length Distribution',
#             'y':0.95,
#             'x':0.5,
#             'xanchor': 'center',
#             'yanchor': 'top',
#             'font': dict(size=24 * text_size_factor, color='black')
#         },
#         xaxis_title={
#             'text': 'Sequence Length (bp)',
#             'font': dict(size=18 * text_size_factor)
#         },
#         yaxis_title={
#             'text': 'Number of Sequences',
#             'font': dict(size=18 * text_size_factor)
#         },
#         template='plotly_white',
#         legend=dict(
#             orientation="h",
#             yanchor="bottom",
#             y=1.02,
#             xanchor="right",
#             x=1,
#             font=dict(size=14 * text_size_factor)
#         ),
#         margin=dict(l=80, r=80, t=120, b=80),
#         annotations=[
#             dict(
#                 xref='paper',
#                 yref='paper',
#                 x=0.99,
#                 y=0.99,
#                 showarrow=False,
#                 text=stats_text,
#                 align='right',
#                 xanchor='right',
#                 yanchor='top',
#                 bgcolor='rgba(255, 255, 255, 0.8)',
#                 bordercolor='grey',
#                 borderwidth=1
#             )
#         ],
#         width=1600, 
#         height=900,
#         font=dict(size=14 * text_size_factor)  # Global font size increase
#     )
    
#     # Configure secondary Y-axis for cumulative distribution
#     if show_cumulative:
#         fig.update_layout(
#             yaxis2=dict(
#                 title={
#                     'text': 'Cumulative Percentage (%)',
#                     'font': dict(size=18 * text_size_factor, color='rgba(219, 64, 82, 0.8)')
#                 },
#                 tickfont=dict(color='rgba(219, 64, 82, 0.8)', size=14 * text_size_factor),
#                 overlaying='y',
#                 side='right',
#                 range=[0, 100]
#             )
#         )
    
#     # Increase tick font size for better readability
#     fig.update_xaxes(tickfont=dict(size=14 * text_size_factor))
#     fig.update_yaxes(tickfont=dict(size=14 * text_size_factor))
    
#     # Show and save graph
#     fig.show()
    
#     # Save as PNG and HTML for flexibility
#     # fig.write_image('distribucion_sin_filtrar.png', scale=5, width=1600, height=900)
#     # fig.write_image('bact_distribution_enhanced.png', scale=5, width=1600, height=900)
#     # fig.write_html('bact_distribution_interactive.html', include_plotlyjs='cdn')
    
#     # return fig

In [9]:
# plot_sequence_length_enhanced(df, 'sequence_length', 
#                              color_scheme='Viridis',
#                              show_stats=True, 
#                              show_cumulative=False,
#                              text_size_factor=1.3)

In [10]:
# plot_sequence_length_enhanced(dfcopia, 'sequence_length', 
#                              color_scheme='Viridis',
#                              show_stats=True, 
#                              show_cumulative=False,
#                              text_size_factor=1.3)

In [11]:

#Limpieza de las secuencias que no tengan los nucleótidos ACGT
base=set('ACGT')
drop_idx=[]
for i,j in enumerate(df['sequence']):
    if set(j)!=base:
        drop_idx.append(i)
print(len(drop_idx))
df=df.drop(drop_idx)
df=df.reset_index()

547


In [12]:
len(df)

38628

In [13]:
# Esto se hizo para explorar los datos y conteo de datos por género

conteo={'genus':[],'conteo':[]}
for i in df['genus'].unique():
    conteo['genus'].append(i)
    conteo['conteo'].append(len(df[df['genus']==i]))

df_conteo=pd.DataFrame(conteo)

temp=df_conteo['conteo']>=100
df_conteo=df_conteo.loc[temp]
df_conteo=df_conteo.reset_index(drop=True)

genes=df_conteo['genus'].tolist()

In [14]:
for g in genes:
    print(f">>>{g}")

>>>Flavobacterium
>>>Vibrio
>>>Corynebacterium
>>>Pseudomonas_E
>>>Pelagibacter
>>>Bradyrhizobium
>>>Mycobacterium
>>>Nocardioides
>>>Streptomyces
>>>Prevotella
>>>Prochlorococcus_A
>>>Streptococcus
>>>Bifidobacterium
>>>Novosphingobium
>>>Pedobacter
>>>Chryseobacterium
>>>Micromonospora
>>>Nocardia
>>>Arthrobacter
>>>Polynucleobacter
>>>Pelagibacter_A
>>>Collinsella
>>>Acinetobacter
>>>Mesorhizobium
>>>Microbacterium
>>>Methylobacterium
>>>Rhizobium
>>>Paracoccus
>>>Paraburkholderia
>>>Sphingomonas


In [15]:
df

,index,sequence_id,sequence,sequence_length,gc_content,domain,phylum,class,order,family,genus,species
0,0,RS_GCF_000657795.2,CTGAAGAGTTTGATCCTGGCTCAGATTGAACGCTGGCGGGATGCTT...,1528,55.562827,__Bacteria,__Pseudomonadota,__Gammaproteobacteria,__Burkholderiales,__Burkholderiaceae,Bordetella,Bordetella
1,1,RS_GCF_000019185.1,ATTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCC...,1542,54.409857,__Bacteria,__Pseudomonadota,__Gammaproteobacteria,__Enterobacterales,__Shewanellaceae,Shewanella,Shewanella
2,2,RS_GCF_004570605.1,CATGAGAGTTTGATCCTGGCTCAGGACAAACGCTGGCGGCGTGCCT...,1547,52.230123,__Bacteria,__Bacillota,__Bacilli,__Lactobacillales,__Aerococcaceae,WM01,WM01
3,3,RS_GCF_002245655.1,CTTAAGAGTTTGATCCTGGCTCAGATTGAACGCTGGCGGCATGCTT...,1536,56.119792,__Bacteria,__Pseudomonadota,__Gammaproteobacteria,__Burkholderiales,__Rhodocyclaceae,Thauera,Thauera
4,4,RS_GCF_016863255.1,GTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCT...,1518,60.276680,__Bacteria,__Actinomycetota,__Actinomycetes,__Mycobacteriales,__Micromonosporaceae,Planosporangium,Planosporangium
...,...,...,...,...,...,...,...,...,...,...,...,...
38623,39170,RS_GCF_004348725.1,ACGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTT...,1517,58.404746,__Bacteria,__Actinomycetota,__Actinomycetes,__Propionibacteriales,__Kribbellaceae,Kribbella,Kribbella
38624,39171,RS_GCF_003097655.1,CTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCT...,1533,53.620352,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium
38625,39172,GB_GCA_016714985.1,ACGGAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGGAGGCCT...,1525,56.327869,__Bacteria,__Bacteroidota,__Bacteroidia,__Chitinophagales,__Saprospiraceae,UBA6168,UBA6168
38626,39173,GB_GCA_030603665.1,ATTAAAGCTCCGGCGCTTCGGGATGGGCCTGCGGCCGATTAGCTAG...,1305,56.551724,__Bacteria,__Chloroflexota,__Anaerolineae,__UBA7937,__B3-Chlor,JAUVXJ01,JAUVXJ01


In [16]:
# # Filtro para los géneros que tienen más de 250 represetantes, se limitó a máximo 250 secuencias por género
# templist=[]
# for i in genes:
#     temp=df[df['genus']==i]
#     temp=temp.sort_values('sequence_length', ascending=False)
#     if len(temp)>250:
#         temp=temp[0:250]
#     templist.append(temp)
# # df_original=df.copy()
# df=pd.concat(templist)
# df=df.reset_index(drop=True)


In [17]:
# Esto se hizo para explorar los datos y conteo de datos por género
conteo={'genus':[],'conteo':[]}
for i in df['genus'].unique():
    conteo['genus'].append(i)
    conteo['conteo'].append(len(df[df['genus']==i]))

df_conteo=pd.DataFrame(conteo)

temp=df_conteo['conteo']>=100
df_conteo=df_conteo.loc[temp]
df_conteo=df_conteo.reset_index(drop=True)

genes=df_conteo['genus'].tolist()

# Filtro para los géneros que tienen más de 100 representantes
# Se separa en datos de entrenamiento (80-120 secuencias) y evaluación (resto)
templist_train = []
templist_eval = []

for i in genes:
    temp = df[df['genus']==i]
    temp = temp.sort_values('sequence_length', ascending=False)
    
    # Determinar cuántas secuencias usar para entrenamiento
    total_sequences = len(temp)
    
    if total_sequences >= 120:
        # Si tiene 120 o más, tomar 120 para entrenamiento
        train_size = 120
    elif total_sequences >= 80:
        # Si tiene entre 80-119, tomar todas para entrenamiento
        train_size = total_sequences
    else:
        # Si tiene menos de 80, tomar todas para entrenamiento (aunque sean pocas)
        train_size = total_sequences
    
    # Separar en entrenamiento y evaluación
    temp_train = temp.iloc[:train_size].copy()
    temp_eval = temp.iloc[train_size:].copy()
    
    # Marcar como datos de entrenamiento
    temp_train['is_training'] = True
    
    # Marcar como datos de evaluación (si existen)
    if len(temp_eval) > 0:
        temp_eval['is_training'] = False
        templist_eval.append(temp_eval)
    
    templist_train.append(temp_train)
    
    print(f"{i}: Total={total_sequences}, Entrenamiento={len(temp_train)}, Evaluación={len(temp_eval)}")

# Concatenar todos los datos
df_train = pd.concat(templist_train)
if templist_eval:  # Solo si hay datos de evaluación
    df_eval = pd.concat(templist_eval)
    df_final = pd.concat([df_train, df_eval])
else:
    df_final = df_train

df_final = df_final.reset_index(drop=True)

# Resumen de la división
total_sequences = len(df_final)
training_sequences = len(df_final[df_final['is_training'] == True])
evaluation_sequences = len(df_final[df_final['is_training'] == False])

print(f"\n=== RESUMEN DE LA DIVISIÓN ===")
print(f"Total de secuencias: {total_sequences}")
print(f"Secuencias de entrenamiento: {training_sequences}")
print(f"Secuencias de evaluación: {evaluation_sequences}")
print(f"Porcentaje entrenamiento: {training_sequences/total_sequences*100:.1f}%")
print(f"Porcentaje evaluación: {evaluation_sequences/total_sequences*100:.1f}%")

# Verificar la distribución por género
print(f"\n=== DISTRIBUCIÓN POR GÉNERO ===")
genus_summary = df_final.groupby(['genus', 'is_training']).size().unstack(fill_value=0)
genus_summary['Total'] = genus_summary.sum(axis=1)
genus_summary = genus_summary.sort_values('Total', ascending=False)
print(genus_summary)

# Asignar el resultado final
df_final

Flavobacterium: Total=365, Entrenamiento=120, Evaluación=245
Vibrio: Total=183, Entrenamiento=120, Evaluación=63
Corynebacterium: Total=201, Entrenamiento=120, Evaluación=81
Pseudomonas_E: Total=680, Entrenamiento=120, Evaluación=560
Pelagibacter: Total=777, Entrenamiento=120, Evaluación=657
Bradyrhizobium: Total=229, Entrenamiento=120, Evaluación=109
Mycobacterium: Total=377, Entrenamiento=120, Evaluación=257
Nocardioides: Total=169, Entrenamiento=120, Evaluación=49
Streptomyces: Total=922, Entrenamiento=120, Evaluación=802
Prevotella: Total=105, Entrenamiento=105, Evaluación=0
Prochlorococcus_A: Total=233, Entrenamiento=120, Evaluación=113
Streptococcus: Total=463, Entrenamiento=120, Evaluación=343
Bifidobacterium: Total=128, Entrenamiento=120, Evaluación=8
Novosphingobium: Total=102, Entrenamiento=102, Evaluación=0
Pedobacter: Total=112, Entrenamiento=112, Evaluación=0
Chryseobacterium: Total=172, Entrenamiento=120, Evaluación=52
Micromonospora: Total=143, Entrenamiento=120, Evaluac

,index,sequence_id,sequence,sequence_length,gc_content,domain,phylum,class,order,family,genus,species,is_training
0,26776,GB_GCA_004296145.1,GGGTTTGATCCTGGCTCAGAATGAACGCTTGTGATATGCTTTACAC...,1538,42.197659,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
1,26798,RS_GCF_027111315.2,TGAAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGGCTTA...,1534,50.847458,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
2,39171,RS_GCF_003097655.1,CTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCT...,1533,53.620352,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
3,32847,GB_GCA_016721735.1,ATGAAGAGTTTGATCCCTGGCTCAGGATGAACGCCAGCGGCAGGCT...,1527,51.669941,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
4,5725,RS_GCF_008806775.1,GTGGAGGGTTCGATTCTGGCTCAGGATGAACGCTGGCGGCGTGCTT...,1525,59.409836,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,11673,RS_GCF_017304635.1,CTTGAGAGTTTGATCATGGCTCAGAATGAACGCTGGCGGCATGCCT...,1426,53.856942,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7348,12778,RS_GCF_014892665.1,TAGTGGCGCACGGGTGCGTAACGCGTGGGAATCTGCCCCTTGGTTC...,1408,54.403409,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7349,18416,RS_GCF_021043375.1,TAGTGGCGCACGGGTGCGTAACGCGTGGGAATCTGCCCCTTGGTTC...,1406,54.551920,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7350,17002,RS_GCF_012650175.1,CTTGAGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCATGCCT...,1389,54.427646,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False


In [26]:
# Se exportan los datos filtrados del df hacia un archivo fasta para utilizar 
# el programa MAFFT para el alineamiento de las secuencias
export_to_fasta(df_final, "datos/fasta_final.fasta")

In [27]:
# Lectura del archivo generado por el programa MFFT
# mafft --auto --thread 12 --maxiterate 1000 --localpair fasta_final.fasta > fasta_final_alineado.fasta
df_imported = read_exported_fasta("datos/fasta_final_alineado.fasta")

In [28]:
df_imported

,sequence_id,sequence,sequence_length,gc_content,genus
0,GB_GCA_004296145.1,----------------------------------------------...,1538,42.197659,Flavobacterium
1,RS_GCF_027111315.2,----------------------------------------------...,1534,50.847458,Flavobacterium
2,RS_GCF_003097655.1,----------------------------------------------...,1533,53.620352,Flavobacterium
3,GB_GCA_016721735.1,----------------------------------------------...,1527,51.669941,Flavobacterium
4,RS_GCF_008806775.1,----------------------------------------------...,1525,59.409836,Flavobacterium
...,...,...,...,...,...
7347,RS_GCF_017304635.1,----------------------------------------------...,1426,53.856942,Sphingomonas
7348,RS_GCF_014892665.1,----------------------------------------------...,1408,54.403409,Sphingomonas
7349,RS_GCF_021043375.1,----------------------------------------------...,1406,54.551920,Sphingomonas
7350,RS_GCF_012650175.1,----------------------------------------------...,1389,54.427646,Sphingomonas


In [29]:
len(df_imported['sequence'][0])

3369

In [38]:
df_final=duckdb.sql("""
select * from df_imported d
join df_final on d.sequence_id= df_final.sequence_id
""").to_df()

In [41]:
df_final

,sequence_id,sequence,sequence_length,gc_content,genus,index,sequence_id_1,sequence_1,sequence_length_1,gc_content_1,domain,phylum,class,order,family,genus_1,species,is_training
0,GB_GCA_004296145.1,----------------------------------------------...,1538,42.197659,Flavobacterium,26776,GB_GCA_004296145.1,GGGTTTGATCCTGGCTCAGAATGAACGCTTGTGATATGCTTTACAC...,1538,42.197659,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
1,RS_GCF_027111315.2,----------------------------------------------...,1534,50.847458,Flavobacterium,26798,RS_GCF_027111315.2,TGAAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGGCTTA...,1534,50.847458,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
2,RS_GCF_003097655.1,----------------------------------------------...,1533,53.620352,Flavobacterium,39171,RS_GCF_003097655.1,CTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCT...,1533,53.620352,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
3,GB_GCA_016721735.1,----------------------------------------------...,1527,51.669941,Flavobacterium,32847,GB_GCA_016721735.1,ATGAAGAGTTTGATCCCTGGCTCAGGATGAACGCCAGCGGCAGGCT...,1527,51.669941,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
4,RS_GCF_008806775.1,----------------------------------------------...,1525,59.409836,Flavobacterium,5725,RS_GCF_008806775.1,GTGGAGGGTTCGATTCTGGCTCAGGATGAACGCTGGCGGCGTGCTT...,1525,59.409836,__Bacteria,__Bacteroidota,__Bacteroidia,__Flavobacteriales,__Flavobacteriaceae,Flavobacterium,Flavobacterium,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,RS_GCF_017304635.1,----------------------------------------------...,1426,53.856942,Sphingomonas,11673,RS_GCF_017304635.1,CTTGAGAGTTTGATCATGGCTCAGAATGAACGCTGGCGGCATGCCT...,1426,53.856942,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7348,RS_GCF_014892665.1,----------------------------------------------...,1408,54.403409,Sphingomonas,12778,RS_GCF_014892665.1,TAGTGGCGCACGGGTGCGTAACGCGTGGGAATCTGCCCCTTGGTTC...,1408,54.403409,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7349,RS_GCF_021043375.1,----------------------------------------------...,1406,54.551920,Sphingomonas,18416,RS_GCF_021043375.1,TAGTGGCGCACGGGTGCGTAACGCGTGGGAATCTGCCCCTTGGTTC...,1406,54.551920,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7350,RS_GCF_012650175.1,----------------------------------------------...,1389,54.427646,Sphingomonas,17002,RS_GCF_012650175.1,CTTGAGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCATGCCT...,1389,54.427646,__Bacteria,__Pseudomonadota,__Alphaproteobacteria,__Sphingomonadales,__Sphingomonadaceae,Sphingomonas,Sphingomonas,False


In [46]:
df_final=df_final[['sequence_id', 'sequence', 'sequence_1',  'sequence_length', 'domain', 'phylum', 'class', 'order', 'family','genus', 'species', 'is_training']]

In [48]:
df_final=df_final.rename(columns={
    "sequence_1":'original_sequence'
})

In [50]:
df_final_copia=df_final.copy()

In [52]:
limpiar_col=['domain', 'phylum', 'class', 'order', 'family']

for c in limpiar_col:
    for i,v in enumerate(df_final[c]):
        df_final.loc[i,c]=v.replace('__', '')

In [55]:
df_final

,sequence_id,sequence,original_sequence,sequence_length,domain,phylum,class,order,family,genus,species,is_training
0,GB_GCA_004296145.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,GGGTTTGATCCTGGCTCAGAATGAACGCTTGTGATATGCTTTACAC...,1538,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium,True
1,RS_GCF_027111315.2,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TGAAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGGCTTA...,1534,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium,True
2,RS_GCF_003097655.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTGAAGAGTTTGATCATGGCTCAGATTGAACGCTGGCGGCAGGCCT...,1533,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium,True
3,GB_GCA_016721735.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,ATGAAGAGTTTGATCCCTGGCTCAGGATGAACGCCAGCGGCAGGCT...,1527,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium,True
4,RS_GCF_008806775.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,GTGGAGGGTTCGATTCTGGCTCAGGATGAACGCTGGCGGCGTGCTT...,1525,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,Flavobacteriaceae,Flavobacterium,Flavobacterium,True
...,...,...,...,...,...,...,...,...,...,...,...,...
7347,RS_GCF_017304635.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTTGAGAGTTTGATCATGGCTCAGAATGAACGCTGGCGGCATGCCT...,1426,Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7348,RS_GCF_014892665.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAGTGGCGCACGGGTGCGTAACGCGTGGGAATCTGCCCCTTGGTTC...,1408,Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7349,RS_GCF_021043375.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,TAGTGGCGCACGGGTGCGTAACGCGTGGGAATCTGCCCCTTGGTTC...,1406,Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,Sphingomonas,False
7350,RS_GCF_012650175.1,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...,CTTGAGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCATGCCT...,1389,Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,Sphingomonas,False


In [54]:
# Se reemplazan los gaps generados por el alineamiento por la letra N
df_final['sequence']=[i.replace('-','N').upper() for i in df_final['sequence']]

In [57]:
# Guardado del archivo ya procesado y listo para los modelos
df_final.to_csv('datos/datos_filtrados_sin_encoding.csv', index=False)